In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import math
import skimage.io as imutils
%matplotlib inline
from keras.models import load_model
from keras.preprocessing import image as keras_preprocess

In [ ]:
#Image structure = > (Height, Width, Color_Channel)
cat_image = Image.open('custom_test/dog.3.jpg')
#Load the Classifier
classifier = load_model('my_model.h5')

# Returns the labels for the classes according to the folder structre of clases
def get_labels_for_clases():
    #return ['car', 'cat', 'dog', 'shoe']
    return ['car' ,'cat', 'dog']

# Run prediction for a single image
def predict_for_single_image(image):
    #lable the images according the folder structure

    lables = get_labels_for_clases()
    out = classifier.predict_classes(image, verbose=0)
    return lables[out[0]]

In [ ]:
#save the image cordinates in a dictionary
image_cordinates_grid = {}

image_dimention_width = cat_image.size[0]
image_dimention_height = cat_image.size[1]

#if 100 x 100 window were to slide thought the image, identify the parameters for the loop condition
slide_window_height_and_width = (600, 600)

#Convert the image in to an numpy array
image_array = np.array(cat_image)

row_count_after_substraction = math.ceil(image_array.shape[0] / slide_window_height_and_width[0])
col_count_after_substraction = math.ceil(image_array.shape[1] / slide_window_height_and_width[1])

#Initialize x, y, w, h values
x, y, w, h = 0, 0, slide_window_height_and_width[1], slide_window_height_and_width[0]
count = 0
print("Image will be transformed in to a", row_count_after_substraction, "x", col_count_after_substraction, "grid")
for row in range(0, row_count_after_substraction):
    for col in range(0, col_count_after_substraction):
        
        #Change the X and Y locations respectivly to add the sliding window effect.
        cropped_image = image_array[y : y + h, x : x + w]
        imutils.imsave("custom_test/cutted/" + str(count) + ".jpg",cropped_image)
        image_cordinates_grid['rect' + str(count)] = (x, y, w, h)

        cropped_image = keras_preprocess.load_img('custom_test/cutted/' + str(count) + ".jpg", target_size=((64, 64)))
        cropped_image = keras_preprocess.img_to_array(cropped_image)
        cropped_image = np.expand_dims(cropped_image, axis=0)
        image_cordinates_grid['prediction' + str(count)] = predict_for_single_image(cropped_image)
        x = x + w
        count = count + 1
    x = 0
    y = y + h

In [ ]:
image_cordinates_grid